이번 Lab을 통해 XOR 문제를 단층 퍼셉트론으로 구현해보겠습니다!

In [1]:
import torch
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

XOR 문제를 풀기 위한 입력과 출력을 정의해줍니다.

In [11]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

이제 1개의 뉴런을 가지는 단층 퍼셉트론을 구현해봅시다. 

In [12]:
# nn layers
linear = nn.Linear(4,4,bias=True)#이 부분을 채워넣으세요, #이 부분을 채워넣으세요, bias=True) # input과 ouput의 크기를 생각해보세요.
sigmoid = nn.Sigmoid() #이 부분을 채워넣으세요 # 시그모이드 함수를 사용하겠습니다.

In [13]:
# model
model = model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)

비용 함수와 옵타마이저를 선언합니다.

In [14]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1# SGD optimizer를 사용하고 learning rate는 1로 하겠습니다.

총 10,001번의 에포크를 수행합니다.

In [16]:
for step in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    if step % 100 == 0:
        print(step, cost.item())

0 0.7471957802772522
100 0.6931002736091614
200 0.6930927634239197
300 0.6930843591690063
400 0.6930747032165527
500 0.6930636167526245
600 0.6930505633354187
700 0.6930350661277771
800 0.6930164098739624
900 0.6929934620857239
1000 0.6929648518562317
1100 0.6929284334182739
1200 0.6928808093070984
1300 0.692816972732544
1400 0.6927281022071838
1500 0.6925994157791138
1600 0.6924024224281311
1700 0.6920791864395142
1800 0.691494345664978
1900 0.6902726888656616
2000 0.687044620513916
2100 0.6737460494041443
2200 0.568760871887207
2300 0.378400981426239
2400 0.013144157826900482
2500 0.00564641784876585
2600 0.0034359795972704887
2700 0.002421896904706955
2800 0.0018505554180592299
2900 0.0014876871136948466
3000 0.0012384598376229405
3100 0.0010574811603873968
3200 0.0009205796523019671
3300 0.0008136135875247419
3400 0.0007279239944182336
3500 0.0006578388856723905
3600 0.0005994782550260425
3700 0.0005502158892340958
3800 0.0005081122508272529
3900 0.0004717650299426168
4000 0.000440

200번 에포크에 비용이 0.6931471824645996가 출력된 이후에는 10,000번 에포크가 되는 순간까지 더 이상 비용이 줄어들지 않습니다. 이는 단층 퍼셉트론은 XOR 문제를 풀 수 없기 때문입니다.

총 10,001회 학습한 단층 퍼셉트론의 예측값도 확인해보겠습니다.

In [17]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[7.122504e-05]
 [9.999286e-01]
 [9.999143e-01]
 [9.548273e-05]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


실제값은 0, 1, 1, 0임에도 예측값은 0, 0, 0, 0으로 문제를 풀지 못하는 모습을 보여줍니다.